In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
W, H = 900, 250

canvas = np.ones((H, W, 3), dtype=np.uint8) * 255

drawing = False
last_x, last_y = None, None

pen_thickness = 18
pen_color = (0, 0, 0)  

In [ ]:
def mouse_draw(event, x, y, flags, param):
    global drawing, last_x, last_y, canvas

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        last_x, last_y = x, y

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            cv2.line(canvas, (last_x, last_y), (x, y), pen_color, pen_thickness, lineType=cv2.LINE_AA)
            last_x, last_y = x, y

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        last_x, last_y = None, None
